In [38]:
import pandas as pd
import json 
from IPython.display import display, HTML
df = pd.read_csv("data/trainingdataset_optimized.csv")

In [5]:
print(df.columns)

Index(['Input_Message', 'Input_JSON', 'Output_Message', 'Output_JSON', 'Datum',
       'Similarity_Score', 'Input_timesort', 'json_timesort'],
      dtype='object')


In [9]:
def display_jsons_side_by_side(input_json, output_json, json_timesort):
    input_pretty = json.dumps(input_json, indent=2, ensure_ascii=False)
    output_pretty = json.dumps(output_json, indent=2, ensure_ascii=False)
    timesort_pretty = json.dumps(json_timesort, indent=2, ensure_ascii=False)
    
    html = f"""
    <div style="display: flex; gap: 20px; font-family: monospace;">
        <div style="flex: 1; white-space: pre; border: 1px solid #ccc; padding: 10px; overflow: auto; max-height: 400px;">
            <h3>Input JSON</h3>
            <pre>{input_pretty}</pre>
        </div>
        <div style="flex: 1; white-space: pre; border: 1px solid #ccc; padding: 10px; overflow: auto; max-height: 400px;">
            <h3>Output JSON</h3>
            <pre>{output_pretty}</pre>
        </div>
    </div>
        <div style="display: flex; gap: 20px; font-family: monospace;">
        <div style="flex: 1; white-space: pre; border: 1px solid #ccc; padding: 10px; overflow: auto; max-height: 400px;">
            <h3>Timesort JSON</h3>
            <pre>{timesort_pretty}</pre>
        </div>
    </div>
    """
    display(HTML(html))

In [10]:
df_random_sample = df.sample(n=1, random_state=42)

display_jsons_side_by_side(
    input_json=json.loads(df_random_sample['Input_JSON'].values[0]),
    output_json=json.loads(df_random_sample['Output_JSON'].values[0]),
    json_timesort=json.loads(df_random_sample['json_timesort'].values[0])
)



In [13]:
df_filtered = df[df['Similarity_Score'] > 0.7]
print(f"Total number of rows in the dataset: {len(df)}")
print(f"Number of rows with semantic similarity > 0.8: {len(df_filtered)}")

Total number of rows in the dataset: 22112
Number of rows with semantic similarity > 0.8: 11313


In [14]:
best_examples = df_filtered.nlargest(5, 'Similarity_Score')
print("Top 5 examples with highest similarity scores:")
for index, row in best_examples.iterrows():
    print(f"Index: {index}, Similarity Score: {row['Similarity_Score']}")
    display_jsons_side_by_side(
        input_json=json.loads(row['Input_JSON']),
        output_json=json.loads(row['Output_JSON']),
        json_timesort=json.loads(row['json_timesort'])
    )

Top 5 examples with highest similarity scores:
Index: 20917, Similarity Score: 0.985576868057251


Index: 18273, Similarity Score: 0.9794912338256836


Index: 9230, Similarity Score: 0.9749243259429932


Index: 19562, Similarity Score: 0.9746868014335632


Index: 20907, Similarity Score: 0.9714138507843018


In [ ]:

import pandas as pd
import re
import arrow
import io
import json

def remove_boilerplate(text: str) -> str:
    """Removes common boilerplate and noisy phrases from a text string."""
    if not isinstance(text, str):
        return text

    boilerplate_phrases = [
        "Več o delovnih zaporah v prometni napovedi.",
        "Več o delovnih zaporah v prometni napovedi",
        "Vir: DARS", 
        "Vir: promet.si",
        "Več na spletni strani...",
        "Več informacij na...",
        # Add any other common phrases you discover
    ]
    
    # Remove null characters and other unwanted artifacts first
    processed_text = text.replace('\u0000', '')
    
    # Remove the boilerplate phrases
    for phrase in boilerplate_phrases:
        # Use re.sub for case-insensitive replacement and to handle potential surrounding whitespace
        processed_text = re.sub(re.escape(phrase), '', processed_text, flags=re.IGNORECASE)
        
    # Clean up any resulting multiple spaces or leading/trailing whitespace
    return " ".join(processed_text.split())

def remove_boilerplate_from_json(json_obj: dict) -> dict:
    """Recursively removes boilerplate from all string values in a JSON object."""
    if not isinstance(json_obj, dict):
        return json_obj
        
    cleaned_obj = {}
    for key, value in json_obj.items():
        if isinstance(value, str):
            cleaned_obj[key] = remove_boilerplate(value)
        elif isinstance(value, dict):
            cleaned_obj[key] = remove_boilerplate_from_json(value) # Recurse for nested objects
        else:
            cleaned_obj[key] = value # Keep non-string, non-dict values as is
    return cleaned_obj

In [ ]:
import pandas as pd
import re
import arrow
import io
import json

# --- Helper Dictionaries for Slovenian Language ---
SLO_MONTHS = {
    'januar': 1, 'januarja': 1, 'februar': 2, 'februarja': 2, 'marec': 3, 'marca': 3,
    'april': 4, 'aprila': 4, 'maj': 5, 'maja': 5, 'junij': 6, 'junija': 6, 'julij': 7,
    'julija': 7, 'avgust': 8, 'avgusta': 8, 'september': 9, 'septembra': 9,
    'oktober': 10, 'oktobra': 10, 'november': 11, 'novembra': 11, 'december': 12, 'decembra': 12
}

def remove_boilerplate(text: str) -> str:
    """Removes common boilerplate and noisy phrases from a text string."""
    if not isinstance(text, str): return text
    boilerplate_phrases = [
        "Več o delovnih zaporah v prometni napovedi.", "Več o delovnih zaporah v prometni napovedi",
        "Vir: DARS", "Vir: promet.si", "Več na spletni strani...", "Več informacij na..."
    ]
    processed_text = text.replace('\u0000', '')
    for phrase in boilerplate_phrases:
        processed_text = re.sub(re.escape(phrase), '', processed_text, flags=re.IGNORECASE)
    return " ".join(processed_text.split())

def remove_boilerplate_from_json(json_obj: dict) -> dict:
    if not isinstance(json_obj, dict): return json_obj
    cleaned_obj = {}
    for key, value in json_obj.items():
        if isinstance(value, str): cleaned_obj[key] = remove_boilerplate(value)
        elif isinstance(value, dict): cleaned_obj[key] = remove_boilerplate_from_json(value)
        else: cleaned_obj[key] = value
    return cleaned_obj

def minimal_date_replacement(text: str, base_time: arrow.Arrow) -> str:
    """
    Replaces explicit dates with relative terms (danes, jutri, včeraj, pojutrišnjem)
    ONLY if they fall within the -1 to +2 day range. All other dates are left untouched.
    """
    if not isinstance(text, str):
        return text

    # Dictionary to map day differences to Slovenian words
    relative_day_map = {
        -1: 'včeraj',
        0: 'danes',
        1: 'jutri',
        2: 'pojutrišnjem'
    }

    # Define patterns to find explicit dates
    # Pattern 1: Day and numeric month (e.g., "1. 12.", "2. 5. ")
    # The negative lookahead `(?!\s*ur[ei])` is crucial to avoid matching time.
    date_pattern_numeric = r'\b(\d{1,2})\.\s*(\d{1,2})\.(?!\s*ur[ei])'
    
    # Pattern 2: Day and month name (e.g., "1. decembra")
    month_names_regex = '|'.join(SLO_MONTHS.keys())
    date_pattern_textual = fr'\b(\d{{1,2}})\.\s*({month_names_regex})\b'

    # Function to perform the replacement logic for a given match
    def replacer(match):
        day_str, month_str = match.groups()
        day = int(day_str)
        
        # Determine month from either numeric or textual pattern
        try:
            month = int(month_str)
        except ValueError:
            month = SLO_MONTHS.get(month_str.lower())
        
        if month is None: # Should not happen with the regex but is safe
            return match.group(0)

        try:
            # Create the target date object. Assume current year.
            target_date = base_time.replace(month=month, day=day)
            
            # If the target month is much earlier than the base month, it's likely next year's date
            if base_time.month == 12 and target_date.month == 1:
                target_date = target_date.shift(years=+1)

            # Calculate the difference in days
            day_diff = (target_date.floor('day') - base_time.floor('day')).days
            
            # The "Three-Day Rule": Only replace if it's a close relative date
            if day_diff in relative_day_map:
                return relative_day_map[day_diff]
            else:
                # Otherwise, return the original matched text, unchanged.
                return match.group(0)

        except (ValueError, TypeError):
            # If date is invalid (e.g., 31. februarja), leave it unchanged
            return match.group(0)

    # Apply the replacement logic
    processed_text = re.sub(date_pattern_numeric, replacer, text, flags=re.IGNORECASE)
    processed_text = re.sub(date_pattern_textual, replacer, processed_text, flags=re.IGNORECASE)
    
    return processed_text


def normalize_json_object(json_obj: dict, base_time: arrow.Arrow) -> dict:
    if not isinstance(json_obj, dict):
        return json_obj
    normalized_obj = {}
    for key, value in json_obj.items():
        if isinstance(value, str):
            normalized_obj[key] = minimal_date_replacement(value, base_time)
        else:
            normalized_obj[key] = value
    return normalized_obj

def apply_minimal_normalization(df: pd.DataFrame) -> pd.DataFrame:
    """
    Applies the minimal date replacement logic to the DataFrame.
    """
    norm_input_msgs, norm_output_msgs, norm_input_jsons, norm_output_jsons, norm_json_timesorts, norm_input_timesorts = [], [], [], [], [], []

    for index, row in df.iterrows():
        base_timestamp_str = str(row['Datum'])
        
        try:
            base_time = arrow.get(base_timestamp_str, ["DD-MM-YYYY HH:mm:ss", "YYYY-MM-DD HH:mm:ss"])
        except arrow.parser.ParserError:
            print(f"Warning: Could not parse date at index {index}: '{base_timestamp_str}'. Skipping.")
            norm_input_msgs.append(row['Input_Message'])
            norm_output_msgs.append(row['Output_Message'])
            norm_input_jsons.append(row['Input_JSON'])
            norm_output_jsons.append(row['Output_JSON'])
            norm_input_timesorts.append(row['Input_timesort'])
            norm_json_timesorts.append(row['json_timesort'])
            continue

        # Normalize Text and JSON columns
        norm_input_msgs.append(minimal_date_replacement(row['Input_Message'], base_time))
        norm_output_msgs.append(minimal_date_replacement(row['Output_Message'], base_time))
        norm_input_timesorts.append(minimal_date_replacement(row['Input_timesort'], base_time))
        
        try:
            input_json_obj = json.loads(row['Input_JSON'])
            norm_input_jsons.append(json.dumps(normalize_json_object(input_json_obj, base_time), ensure_ascii=False, indent=2))
        except (json.JSONDecodeError, TypeError):
            norm_input_jsons.append(row['Input_JSON'])

        try:
            output_json_obj = json.loads(row['Output_JSON'])
            norm_output_jsons.append(json.dumps(normalize_json_object(output_json_obj, base_time), ensure_ascii=False, indent=2))
        except (json.JSONDecodeError, TypeError):
            norm_output_jsons.append(row['Output_JSON'])
            
        try:
            json_timesort_obj = json.loads(row['json_timesort'])
            norm_json_timesorts.append(json.dumps(normalize_json_object(json_timesort_obj, base_time), ensure_ascii=False, indent=2))
        except (json.JSONDecodeError, TypeError):
            norm_json_timesorts.append(row['json_timesort'])

    df['Input_Message_Normalized'] = norm_input_msgs
    df['Output_Message_Normalized'] = norm_output_msgs
    df['Input_JSON_Normalized'] = norm_input_jsons
    df['Output_JSON_Normalized'] = norm_output_jsons
    df['input_timesort_Normalized'] = norm_input_timesorts
    df['json_timesort_Normalized'] = norm_json_timesorts
    
    return df


df = best_examples.reset_index(drop=True)
df_normalized = apply_minimal_normalization(df.copy())
#print(df_normalized.head())
# 3. Display the results
pd.set_option('display.max_colwidth', None)
print("--- NORMALIZATION RESULTS FOR ROW 0 ---")
#print("Base Time:", df_normalized.loc[0, 'Datum'])
i = 2
print("\nOriginal Input Message:\n", df_normalized.loc[i, 'Input_Message'])
print("\nNormalized Input Message:\n", df_normalized.loc[i, 'Input_Message_Normalized'])
print("\nOriginal Output Message:\n", df_normalized.loc[i, 'Output_Message'])
print("\nNormalized Output Message:\n", df_normalized.loc[i, 'Output_Message_Normalized'])
print("\nOriginal Input JSON:\n", df_normalized.loc[i, 'Input_JSON'])
print("\nNormalized Input JSON:\n", df_normalized.loc[i, 'Input_JSON_Normalized'])
print("\nOriginal Output JSON:\n", df_normalized.loc[i, 'Output_JSON'])
print("\nNormalized Output JSON:\n", df_normalized.loc[i, 'Output_JSON_Normalized'])

--- NORMALIZATION RESULTS FOR ROW 0 ---

Original Input Message:
 Na ljubljanski severni obvoznici med krožiščem Tomačevo in razcepom Zadobrova je promet urejen po dveh zoženih pasovih v obe smeri, zaprt je izvoz Nove Jarše iz smeri Kosez. Na gorenjski avtocesti med galerijo Moste in priključkom Podtabor promet v obe smeri poteka po dveh zoženih pasovih. Zaprt je priključek Brezje proti Ljubljani in priključek Radovljica proti Karavankam, za izvoza Brezje in Lesce proti Karavankam vozite po desnem voznem pasu. - Cesta Novo mesto - Soteska bo zaprta v Novem mestu do 28. aprila. - Cesta Cikava - Grosuplje predvidoma do 30. junija zaprta na Grosupljem. - Predvidoma do 31. maja v Žicah zaprta cesta Žice - Sveta Ana - Zgornja Ščavnica. Več o delovnih zaporah v prometni napovedi. Več o delovnih zaporah v prometni napovedi. Zaradi jutranje prometne konice je povečan in upočasnjen promet na mestnih obvoznicah in cestah, ki vodijo proti mestnim središčem. Proti Ljubljani so zastoji iz smeri Pri

In [35]:
import pandas as pd
import re
import arrow
import io
import json

def remove_boilerplate(text: str) -> str:
    """Removes common boilerplate and noisy phrases from a text string."""
    if not isinstance(text, str): return text
    boilerplate_phrases = [
        "Več o delovnih zaporah v prometni napovedi.", "Več o delovnih zaporah v prometni napovedi",
        "Vir: DARS", "Vir: promet.si", "Več na spletni strani...", "Več informacij na..."
    ]
    processed_text = text.replace('\u0000', '')
    for phrase in boilerplate_phrases:
        processed_text = re.sub(re.escape(phrase), '', processed_text, flags=re.IGNORECASE)
    return " ".join(processed_text.split())

def remove_boilerplate_from_json(json_obj: dict) -> dict:
    if not isinstance(json_obj, dict): return json_obj
    cleaned_obj = {}
    for key, value in json_obj.items():
        if isinstance(value, str): cleaned_obj[key] = remove_boilerplate(value)
        elif isinstance(value, dict): cleaned_obj[key] = remove_boilerplate_from_json(value)
        else: cleaned_obj[key] = value
    return cleaned_obj

def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Applies boilerplate removal and date normalization, preserving all original columns.
    """
    all_processed_rows = [] # This will be a list of dictionaries, one for each row

    for index, row in df.iterrows():
        # --- Create a dictionary for the new row, preserving metadata ---
        new_row_data = {
            # Preserve original and metadata columns
            'Input_Message': row['Input_Message'],
            'Output_Message': row['Output_Message'],
            'Input_JSON': row['Input_JSON'],
            'Output_JSON': row['Output_JSON'],
            'Datum': row['Datum'],
            'Input_timesort': row.get('Input_timesort'), # Use .get for safety
            'json_timesort': row.get('json_timesort'),    # Use .get for safety
            'Similarity_Score': row.get('Similarity_Score', None),  # Preserve similarity score if it exists
        }

        # --- Step A: Boilerplate Removal ---
        clean_input_msg = remove_boilerplate(row['Input_Message'])
        clean_output_msg = remove_boilerplate(row['Output_Message'])
        clean_timesort = remove_boilerplate(row.get('Input_timesort', ''))  # Use .get for safety
        try:
            clean_input_json_obj = remove_boilerplate_from_json(json.loads(row['Input_JSON']))
        except (json.JSONDecodeError, TypeError):
            clean_input_json_obj = {}
        try:
            clean_output_json_obj = remove_boilerplate_from_json(json.loads(row['Output_JSON']))
        except (json.JSONDecodeError, TypeError):
            clean_output_json_obj = {}
        try:
            clean_json_timesort_obj = remove_boilerplate_from_json(json.loads(row.get('json_timesort', '{}')))
        except (json.JSONDecodeError, TypeError):
            clean_json_timesort_obj = {}
        
        new_row_data['Input_Message'] = clean_input_msg
        new_row_data['Output_Message'] = clean_output_msg
        new_row_data['Input_JSON'] = json.dumps(clean_input_json_obj, ensure_ascii=False, indent=2)
        new_row_data['Output_JSON'] = json.dumps(clean_output_json_obj, ensure_ascii=False, indent=2)
        new_row_data['Input_timesort'] = clean_timesort
        new_row_data['json_timesort'] = json.dumps(clean_json_timesort_obj, ensure_ascii=False, indent=2)

        
        all_processed_rows.append(new_row_data)

    # Create the final DataFrame from the list of row dictionaries
    return pd.DataFrame(all_processed_rows)

# --- Example Usage ---

# Create a sample DataFrame with all specified columns
csv_data = """Input_Message,Input_JSON,Output_Message,Output_JSON,Datum,Input_timesort,json_timesort
"Na štajerski avtocesti med Framom in Slovensko Bistrico bodo do nedelje, 1. 12., zapirali pas. Več o delovnih zaporah v prometni napovedi.","{""Delovne zapore"": ""Na štajerski avtocesti bodo do nedelje, 1. 12., zapirali pas. Vir: DARS""}", "Na štajerski avtocesti bodo do jutri zapirali pas.","{""Zastoji"": ""Na štajerski avtocesti bodo do jutri zapirali pas.\u0000""}", "30-11-2024 07:16:00",1,2
"""

df = best_examples.reset_index(drop=True)

# Run the full processing pipeline
df_processed = process_dataframe(df.copy())

# Display the final DataFrame, showing that all columns are preserved
print("--- FINAL PROCESSED DATAFRAME (showing all columns) ---")
# Use .T to transpose the view for better readability of many columns
display(df_processed.T)

--- FINAL PROCESSED DATAFRAME (showing all columns) ---


,0,1,2,3,4
Input_Message,"Štajerska avtocesta bo zaprta med priključkoma Blagovica in Vransko proti Mariboru do ponedeljka, 2. 12. do 5. ure. Na priključku Trojane sta zaprta tudi uvoza proti Mariboru in Ljubljani, obvoz je urejen po regionalni cesti. Pričakujemo zastoje. Na štajerski avtocesti med Framom in Slovensko Bistrico bodo do nedelje, 1. 12., izmenično zapirali vozni in prehitevalni pas v obe smeri zaradi odstranjevanja delovne zapore. Lahko nastane zastoj. na ostalih cestah: - Cesta Koprivna - Črna, pri Pristavi, bo zaprta do 17. ure. - Cesta Litija - Zagorje bo zaprta pri Šklendrovcu danes do 17. ure ter v prihodnjem tednu od ponedeljka do sobote, vsak dan med 8. in 17. uro. - Cesta Bistrica pri Tržiču - Begunje, v Bistrici, na Begunjski cesti, do nedelje do 17. ure. - V Ljubljani bo zaprta Litijska cesta med Pesarsko cesto in Potjo na Breje danes od 8. ure do polnoči. . Na štajerski avtocesti na izvozu Blagovica iz smeri Ljubljane. Na primorski avtocesti je zaradi okvare vozila oviran promet med Vrhniko in Brezovico proti Ljubljani. Na štajerski avtocesti je oviran promet na odstavnem pasu pred izvozom Blagovica proti Mariboru. 30-11-2024 07:16:00","Na štajerski avtocesti bo med Šentjakobom in Domžalami proti Mariboru promet potekal po enem pasu. Zaprt je tudi izvoz Domžale proti Mariboru. Obvoz je preko priključkov Šentjakob oz. Krtina. Na gorenjski avtocesti med priključkoma Jesenice vzhod in Jesenice zahod v obe smeri promet poteka po enem voznem pasu v vsako smer, predvidoma do konca julija. Zaradi sanacije objekta po prometni nesreči je regionalna cesta Čepinci - meja z Madžarsko, na mejnem prehodu Čepinci, zaprta. . 07-04-2024 21:41:00","Na ljubljanski severni obvoznici med krožiščem Tomačevo in razcepom Zadobrova je promet urejen po dveh zoženih pasovih v obe smeri, zaprt je izvoz Nove Jarše iz smeri Kosez. Na gorenjski avtocesti med galerijo Moste in priključkom Podtabor promet v obe smeri poteka po dveh zoženih pasovih. Zaprt je priključek Brezje proti Ljubljani in priključek Radovljica proti Karavankam, za izvoza Brezje in Lesce proti Karavankam vozite po desnem voznem pasu. - Cesta Novo mesto - Soteska bo zaprta v Novem mestu do 28. aprila. - Cesta Cikava - Grosuplje predvidoma do 30. junija zaprta na Grosupljem. - Predvidoma do 31. maja v Žicah zaprta cesta Žice - Sveta Ana - Zgornja Ščavnica. Zaradi jutranje prometne konice je povečan in upočasnjen promet na mestnih obvoznicah in cestah, ki vodijo proti mestnim središčem. Proti Ljubljani so zastoji iz smeri Primorske, Štajerske in Dolenjske. Na pomurski avtocesti pred delovno zaporo pred razcepom Dragučova proti Lendavi, zamuda 5 - 10 minut. Predmet na vozišču ovira promet na vipavski hitri cesti med priključkoma Šempeter in Vogrsko proti Razdrtemu. Na pomurski avtocesti je oviran promet med razcepom Dragučova in priključkom Pernica proti Lendavi, vozila so na odstavnem pasu. Na regionalni cesti Razdrto - Postojna je oviran promet pri Dilcah. 19-04-2023 05:39:00","Primorska avtocesta: - zaprt je prehitevalni pas med Uncem in Logatcem proti Ljubljani, od 12.30 do 17. ure pa še med Logatcem in Uncem proti Kopru; - med Vrhniko in Brezovico proti Ljubljani bo do 17. ure promet potekal po enem prometnem pasu. . Tovorna vozila Danes do 22. ure velja omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7,5 t. Na primorski avtocesti med Vrhniko in Brezovico proti Ljubljani, pred delovno zaporo. Na dolenjski avtocesti je zaradi okvare tovornega vozila zaprt prehitevalni pas v pokritem vkopu Karteljevo proti Novemu mestu. 18-02-2024 11:02:00","Štajerska avtocesta bo zaprta med priključkoma Blagovica in Vransko proti Mariboru do ponedeljka, 2. 12. do 5. ure. Na priključku Trojane sta zaprta tudi uvoza proti Mariboru in Ljubljani, obvoz je urejen po regionalni cesti. Pričakujemo zastoje. Na štajerski avtocesti med Framom in Slovensko Bistrico bodo do nedelje, 1. 12., izmenično zapirali vozni in prehitevalni pas v obe smeri zaradi odstranjevanja

In [39]:
df_filtered = df[df['Similarity_Score'] > 0.7]
df_cleaned = process_dataframe(df_filtered)
df_normalized = apply_minimal_normalization(df_cleaned.copy())
df_normalized.to_csv("data/trainingdataset_optimized_normalized.csv", index=False, encoding='utf-8')



In [40]:
print(df_normalized.columns)
df_normalized = df_normalized.drop(columns=['Input_Message', 'Output_Message', 'Input_JSON', 'Output_JSON', 'Input_timesort', 'json_timesort'])
df_normalized = df_normalized.rename(columns={
    'Input_Message_Normalized': 'Input_Message',
    'Output_Message_Normalized': 'Output_Message',
    'Input_JSON_Normalized': 'Input_JSON',
    'Output_JSON_Normalized': 'Output_JSON',
    'input_timesort_Normalized': 'Input_timesort',
    'json_timesort_Normalized': 'json_timesort'
})
df_normalized.to_csv("data/trainingdataset_optimized_normalized.csv", index=False, encoding='utf-8')

Index(['Input_Message', 'Output_Message', 'Input_JSON', 'Output_JSON', 'Datum',
       'Input_timesort', 'json_timesort', 'Similarity_Score',
       'Input_Message_Normalized', 'Output_Message_Normalized',
       'Input_JSON_Normalized', 'Output_JSON_Normalized',
       'input_timesort_Normalized', 'json_timesort_Normalized'],
      dtype='object')
